# Loading Titanic CSV

In [ ]:
import functools

import numpy as np
import tensorflow as tf

In [ ]:
train_file_path = 'resources/train.csv'
test_file_path = 'resources/eval.csv'

## Load data

In [ ]:
!head {train_file_path}

In [ ]:
LABEL_COLUMN = 'survived'
LABELS = [0, 1]

In [ ]:
def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=5,
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True, 
      **kwargs)
  return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

In [ ]:
def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))

In [ ]:
show_batch(raw_train_data)

In [ ]:
CSV_COLUMNS = [
    'survived',
    'sex',
    'age',
    'n_siblings_spouses',
    'parch',
    'fare',
    'class',
    'deck',
    'embark_town',
    'alone'
]

temp_dataset = get_dataset(
    train_file_path,
    column_names=CSV_COLUMNS
)

show_batch(temp_dataset)

In [ ]:
SELECT_COLUMNS = [
    'survived',
    'age',
    'n_siblings_spouses',
    'class',
    'deck',
    'alone'
]

temp_dataset = get_dataset(
    train_file_path,
    select_columns=SELECT_COLUMNS
)

show_batch(temp_dataset)

### Numerical data

In [ ]:
SELECT_COLUMNS = [
    'survived',
    'age',
    'n_siblings_spouses',
    'parch',
    'fare'
]

DEFAULTS = [
    0,
    0.0,
    0.0,
    0.0,
    0.0
]

temp_dataset = get_dataset(
    train_file_path,
    select_columns=SELECT_COLUMNS,
    column_defaults=DEFAULTS)

show_batch(temp_dataset)

In [ ]:
example_batch, labels_batch = next(iter(temp_dataset)) 

In [ ]:
def pack(features, label):
  return tf.stack(list(features.values()), axis=-1), label

In [ ]:
packed_dataset = temp_dataset.map(pack)

for features, labels in packed_dataset.take(1):
  print(features.numpy())
  print()
  print(labels.numpy())

In [ ]:
show_batch(raw_train_data)

In [ ]:
example_batch, labels_batch = next(iter(temp_dataset)) 

In [ ]:
class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

In [ ]:
NUMERIC_FEATURES = [
    'age',
    'n_siblings_spouses',
    'parch',
    'fare'
]

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

In [ ]:
show_batch(packed_train_data)

In [ ]:
example_batch, labels_batch = next(iter(packed_train_data)) 

In [ ]:
import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

In [ ]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [ ]:
def normalize_numeric_data(data, mean, std):
  return (data - mean) / std

In [ ]:
normalizer = functools.partial(
    normalize_numeric_data,
    mean=MEAN,std=STD
)

numeric_column = tf.feature_column.numeric_column(
    'numeric',
    normalizer_fn=normalizer,
    shape=[len(NUMERIC_FEATURES)]
)

numeric_columns = [numeric_column]
numeric_column

In [ ]:
example_batch['numeric']

In [ ]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

### Categorical data

In [ ]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}

In [ ]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [ ]:
categorical_columns

In [ ]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

### Combined preprocessing layer

In [ ]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns + numeric_columns)

In [ ]:
print(preprocessing_layer(example_batch).numpy()[0])

## Build the model

In [ ]:
model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

## Train, evaluate, and predict

In [ ]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

In [ ]:
model.fit(train_data, epochs=20)

In [ ]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

In [ ]:
predictions = model.predict(test_data)

for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
    print(
        "Predicted survival: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("SURVIVED" if bool(survived) else "DIED")
    )